In [33]:
import os
import pandas as pd

from importlib import reload
import countries
reload(countries)
if os.getcwd().split('/')[-1] == 'mobility': os.chdir('..')
from countries import FROM_COUNTRY

df = pd.read_csv('dist/static/mobility/world.csv.gz')
df

,country_geoid,region,category,page,change,date,value,country
0,AE,total,retail/recreation,0,-55,2020-02-16,0.857313,United Arab Emirates
1,AE,total,retail/recreation,0,-55,2020-02-17,-1.365351,United Arab Emirates
2,AE,total,retail/recreation,0,-55,2020-02-18,-1.621751,United Arab Emirates
3,AE,total,retail/recreation,0,-55,2020-02-19,-1.549515,United Arab Emirates
4,AE,total,retail/recreation,0,-55,2020-02-20,-1.748761,United Arab Emirates
...,...,...,...,...,...,...,...,...
145549,ZW,total,residential,1,13,2020-03-25,15.087301,Zimbabwe
145550,ZW,total,residential,1,13,2020-03-26,18.100492,Zimbabwe
145551,ZW,total,residential,1,13,2020-03-27,23.208374,Zimbabwe
145552,ZW,total,residential,1,13,2020-03-28,13.050313,Zimbabwe


In [6]:
import altair as alt

country = "France"
category = "workplace"
subset = df.query('country == @country and category == @category')
chart_data = subset.groupby(['date'])['value'].mean().reset_index()
(alt.Chart(chart_data)
 .mark_line(point=True)
 .encode(x='date', y='value')
 +
 alt.Chart(subset.query('region == "total"'))
 .mark_line(point=True)
 .encode(x='date', y='value')
)

In [7]:
(alt.Chart(subset.reset_index())
 .mark_line(point=True)
 .encode(x='date', y='value', color='region', tooltip=['region', 'date', 'value'])
 .properties(width=700)
)

In [8]:
fr = df.query('country == "France" and region == "total"')
alt.Chart(fr).mark_line(point=True).encode(x='date:T', y='value', color='category')

In [40]:
df = df.assign(country_code=df['country'].map(FROM_COUNTRY))
df

,country_geoid,region,category,page,change,date,value,country,country_code
0,AE,total,retail/recreation,0,-55,2020-02-16,0.857313,United Arab Emirates,784
1,AE,total,retail/recreation,0,-55,2020-02-17,-1.365351,United Arab Emirates,784
2,AE,total,retail/recreation,0,-55,2020-02-18,-1.621751,United Arab Emirates,784
3,AE,total,retail/recreation,0,-55,2020-02-19,-1.549515,United Arab Emirates,784
4,AE,total,retail/recreation,0,-55,2020-02-20,-1.748761,United Arab Emirates,784
...,...,...,...,...,...,...,...,...,...
145549,ZW,total,residential,1,13,2020-03-25,15.087301,Zimbabwe,716
145550,ZW,total,residential,1,13,2020-03-26,18.100492,Zimbabwe,716
145551,ZW,total,residential,1,13,2020-03-27,23.208374,Zimbabwe,716
145552,ZW,total,residential,1,13,2020-03-28,13.050313,Zimbabwe,716


In [242]:
from vega_datasets import data

world_topo = alt.topo_feature(data.world_110m.url, 'countries')
alt.data_transformers.enable('data_server')

DataTransformerRegistry.enable('data_server')

In [264]:
categories = list(df.category.unique())
map_data = (df
              .query('region == "total"')
              .set_index(['country', 'country_code', 'region', 'category'])
              [['date', 'value']]
            .assign(value=lambda f: f['value'].round(1))
            ['value']
              .groupby(['country', 'country_code', 'region', 'category'])
              .last()
            .unstack()
              .reset_index())

input_dropdown = alt.binding_select(options=categories)
selection_category = alt.selection_single(fields=['category'], 
                                          bind=input_dropdown, 
                                          name='Mobility',
                                          init={'category': 'workplace'})

selection_country = alt.selection_single(
        fields=['country'],
        name='Country of',
        empty='all',
#     init={'country': 'France'}
    )

background = (alt.Chart(world_topo)
              .mark_geoshape(fill='lightgray', stroke='white', strokeWidth=0.5)
              .transform_filter('datum.id != 10')
#               .transform_filter('datum.id != 304')
             )

foreground = (alt.Chart(world_topo)
        .mark_geoshape(stroke='white', strokeWidth=0.5)
         .encode(
             color=alt.condition(
                 selection_country,
                 alt.Color('value:Q', scale=alt.Scale(scheme='blueorange')),
                 alt.value('lightgray')
             ),
             tooltip=['value:Q', 'country:N']
         )
            .transform_lookup(
                lookup='id',
                from_=alt.LookupData(data=map_data,
                                 key='country_code', 
                                 fields=['country'] + categories)
            )
            .transform_fold(
                fold=categories,
                as_=['category', 'value']
            )
             .add_selection(selection_category)
             .transform_filter(selection_category)
            ).add_selection(selection_country)

map_chart = (background + foreground).properties(width=700, height=500)

base_ts = alt.Chart(df.query("region == 'total'").assign(value=lambda f: f['value'].round(1)))

ts_chart = (base_ts
            .mark_line(point=True)
            .encode(x='date:T', y='value:Q', tooltip=['date:T', 'value'])
            .add_selection(selection_category)
            .add_selection(selection_country)
            .transform_filter(selection_category)
            .transform_filter(selection_country)
            .transform_aggregate(
                value='mean(value)',
                groupby=['category', 'date']
            )
           )

chart = map_chart | ts_chart
chart.save('dist/static/mobility/charts/world.json')
chart

In [207]:
df.country.unique()

array(['United Arab Emirates', 'Afghanistan', 'Antigua and Barbuda',
       'Angola', 'Argentina', 'Austria', 'Australia', 'Aruba',
       'Bosnia and Herzegovina', 'Barbados', 'Bangladesh', 'Belgium',
       'Burkina Faso', 'Bulgaria', 'Bahrain', 'Benin', 'Bolivia',
       'Brazil', 'Bahamas', 'Botswana', 'Belarus', 'Belize', 'Canada',
       'Switzerland', 'Cote dIvoire', 'Chile', 'Cameroon', 'Colombia',
       'Costa Rica', 'Cape Verde', 'Czech Republic', 'Germany', 'Denmark',
       'Dominican Republic', 'Ecuador', 'Estonia', 'Egypt', 'Spain',
       'Finland', 'Fiji', 'France', 'Gabon', 'Georgia', 'Ghana',
       'Guatemala', 'Guinea Bissau', 'Honduras', 'Croatia', 'Haiti',
       'Hungary', 'Indonesia', 'Ireland', 'Israel', 'India', 'Iraq',
       'Italy', 'Jamaica', 'Jordan', 'Japan', 'Kenya', 'Kyrgyzstan',
       'Cambodia', 'South Korea', 'Kuwait', 'Kazakhstan', 'Laos',
       'Lebanon', 'Sri Lanka', 'Lithuania', 'Luxembourg', 'Latvia',
       'Libya', 'Moldova', 'North Macedo

In [177]:
ts

,country_geoid,region,category,page,change,date,value,country,country_code
0,AE,total,retail/recreation,0,-55,2020-02-16,0.857313,United Arab Emirates,784
1,AE,total,retail/recreation,0,-55,2020-02-17,-1.365351,United Arab Emirates,784
2,AE,total,retail/recreation,0,-55,2020-02-18,-1.621751,United Arab Emirates,784
3,AE,total,retail/recreation,0,-55,2020-02-19,-1.549515,United Arab Emirates,784
4,AE,total,retail/recreation,0,-55,2020-02-20,-1.748761,United Arab Emirates,784
...,...,...,...,...,...,...,...,...,...
145549,ZW,total,residential,1,13,2020-03-25,15.087301,Zimbabwe,716
145550,ZW,total,residential,1,13,2020-03-26,18.100492,Zimbabwe,716
145551,ZW,total,residential,1,13,2020-03-27,23.208374,Zimbabwe,716
145552,ZW,total,residential,1,13,2020-03-28,13.050313,Zimbabwe,716
